# Home Credit Default Risk

This project is try to use statistic and machine learning method to predict the default risk of Home Cridit's clients. The dataset includes the internal infromation of Home Cridit and external information from other financial institutions. The feature engineering of this project is mainly aggregate the data from different sources and try to create new features based on current features.

Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

## Data

* **application_{train|test}.csv**

This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
Static data for all applications. One row represents one loan in our data sample.
* **bureau.csv**

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.
* **bureau_balance.csv**

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.
* **POS_CASH_balance.csv**

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.
* **credit_card_balance.csv**

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.
* **previous_application.csv**

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.
* **installments_payments.csv**

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.
* **HomeCredit_columns_description.csv**

This file contains descriptions for the columns in the various data files.

## Feature Engineering

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))
print(os.listdir("./"))

# Any results you write to the current directory are saved as output.

['.ipynb_checkpoints', '0.798', '0.8', 'application_test.csv', 'application_test.csv.zip', 'application_train.csv', 'application_train.csv.zip', 'app_data.csv', 'baseline_lgb.csv', 'baseline_lgb2.csv', 'Blend', 'bureau.csv', 'bureau.csv.zip', 'bureau_balance.csv', 'bureau_balance.csv.zip', 'bur_data.csv', 'bur_joined.csv', 'card_data.csv', 'credit_card_balance.csv', 'credit_card_balance.csv.zip', 'data-process-organized.ipynb', 'Data_process.ipynb', 'HomeCredit_columns_description.csv', 'hybridII.csv', 'installments_payments.csv', 'installments_payments.csv.zip', 'inst_data.csv', 'kernel.ipynb', 'LGBT_v1.ipynb', 'POS_CASH_balance.csv', 'POS_CASH_balance.csv.zip', 'pos_data.csv', 'previous_application.csv', 'previous_application.csv.zip', 'sample_submission.csv', 'sample_submission.csv.zip', 'script.py', 'submission0.789.csv', 'submission_0.797.csv', 'submission_kernel02.csv', 'submission_kernel03.csv', 'submission_kernel04.csv', 'submission_xgb0.791.csv', 'test.csv', 'tidy_xgb_0.796.cs

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# pre_bur = pd.read_csv('../input/bureau.csv')
# pre_burbal = pd.read_csv('../input/bureau_balance.csv') 
# pre_app = pd.read_csv('../input/previous_application.csv')
# pre_pos = pd.read_csv('../input/POS_CASH_balance.csv')
# pre_inst = pd.read_csv('../input/installments_payments.csv')
# pre_card = pd.read_csv('../input/credit_card_balance.csv')

# load all the dataset
pre_bur = pd.read_csv('bureau.csv')
pre_burbal = pd.read_csv('bureau_balance.csv') 
pre_app = pd.read_csv('previous_application.csv')
pre_pos = pd.read_csv('POS_CASH_balance.csv')
pre_inst = pd.read_csv('installments_payments.csv')
pre_card = pd.read_csv('credit_card_balance.csv')

**Define data process functions**

In [3]:
# level 2 data process
def process_burbal(df):
    gp = df.groupby('SK_ID_BUREAU')
    loan_rt = lambda x: sum((x!=0)*1)/len(x)
    agg = gp.agg({'STATUS':lambda x: x.mode().iloc[0],
                           'MONTHS_BALANCE':[np.min, np.sum, np.mean, loan_rt]})
    agg.columns = ['STATUS','MONTH_MIN','MONTH_SUM','MONTH_MEAN','LOAN_RATIO']
    return agg

def join_burbal(bur,bal):
    bur = bur.set_index('SK_ID_BUREAU')
    bur_join = bur.join(other=bal,how='inner')
    bur_join = bur_join.reset_index()
    return bur_join.drop('SK_ID_BUREAU',axis=1)

# level 1 data process
def cat_dummies(df,cols):
    for col in cols:
        sub_df = pd.get_dummies(df[col],prefix=col.upper(),dummy_na= True)
        df = pd.concat([df.drop(col,axis=1),sub_df],axis=1)
    return df

def process_obj(df):
    df_obj = df.select_dtypes(include='object')
    cat_cols = []
    bin_cols = []
    for col in df_obj.columns.values:
        if len(df_obj[col].value_counts())<=2:
            bin_cols.append(col)
        elif len(df_obj[col].value_counts())<10:
            cat_cols.append(col)
        else:
            df_obj = df_obj.drop(col,axis=1)
            print('drop col',col,':two many categories')
    df_obj = cat_dummies(df_obj,cat_cols)
    for col in bin_cols:
        df_obj[col],uniques = pd.factorize(df_obj[col])
    df_num = df.select_dtypes(include='number')
    return pd.concat([df_num,df_obj],axis=1)

def group_agg(df,title,col='SK_ID_CURR'):
    gp = df.groupby(col)
    gp_data = gp.agg(['min','max','mean','sum','var'])
    gp_data.columns = pd.Index([title + e[0] + "_" + e[1].upper() for e in gp_data.columns.values])
#     gp_data.columns = [' '.join(col).strip() for col in gp_data.columns.values]
    return gp_data.reset_index()

*Left Bureau Data Process*

In [4]:
burbal = process_burbal(pre_burbal)
bur = join_burbal(pre_bur,pre_burbal)
bur_data = process_obj(bur)
bur_data = group_agg(bur_data,'BUR_')
bur_data.head()

drop col CREDIT_TYPE :two many categories


,SK_ID_CURR,BUR_index_MIN,BUR_index_MAX,BUR_index_MEAN,BUR_index_SUM,BUR_index_VAR,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_SUM,...,BUR_STATUS_X_MIN,BUR_STATUS_X_MAX,BUR_STATUS_X_MEAN,BUR_STATUS_X_SUM,BUR_STATUS_X_VAR,BUR_STATUS_nan_MIN,BUR_STATUS_nan_MAX,BUR_STATUS_nan_MEAN,BUR_STATUS_nan_SUM,BUR_STATUS_nan_VAR
0,100001,5896630,5896636,5896633.000,41276431,4.666667e+00,-1572,-49,-735.000000,-5145,...,0,0,0.0,0,0.0,0,0,0,0,0.0
1,100002,6113835,6158909,6153272.125,49226177,2.539244e+08,-1437,-103,-874.000000,-6992,...,0,0,0.0,0,0.0,0,0,0,0,0.0
2,100003,5885877,5885880,5885878.500,23543514,1.666667e+00,-2586,-606,-1400.750000,-5603,...,0,0,0.0,0,0.0,0,0,0,0,0.0
3,100004,6829133,6829134,6829133.500,13658267,5.000000e-01,-1326,-408,-867.000000,-1734,...,0,0,0.0,0,0.0,0,0,0,0,0.0
4,100005,6735200,6735202,6735201.000,20205603,1.000000e+00,-373,-62,-190.666667,-572,...,0,0,0.0,0,0.0,0,0,0,0,0.0


In [5]:
bur_data = bur_data.drop(['BUR_index_MIN','BUR_index_MAX','BUR_index_MEAN','BUR_index_SUM','BUR_index_VAR'],axis=1)
bur_data.head(5)

,SK_ID_CURR,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_SUM,BUR_DAYS_CREDIT_VAR,BUR_CREDIT_DAY_OVERDUE_MIN,BUR_CREDIT_DAY_OVERDUE_MAX,BUR_CREDIT_DAY_OVERDUE_MEAN,BUR_CREDIT_DAY_OVERDUE_SUM,...,BUR_STATUS_X_MIN,BUR_STATUS_X_MAX,BUR_STATUS_X_MEAN,BUR_STATUS_X_SUM,BUR_STATUS_X_VAR,BUR_STATUS_nan_MIN,BUR_STATUS_nan_MAX,BUR_STATUS_nan_MEAN,BUR_STATUS_nan_SUM,BUR_STATUS_nan_VAR
0,100001,-1572,-49,-735.000000,-5145,240043.666667,0,0,0.0,0,...,0,0,0.0,0,0.0,0,0,0,0,0.0
1,100002,-1437,-103,-874.000000,-6992,186150.000000,0,0,0.0,0,...,0,0,0.0,0,0.0,0,0,0,0,0.0
2,100003,-2586,-606,-1400.750000,-5603,827783.583333,0,0,0.0,0,...,0,0,0.0,0,0.0,0,0,0,0,0.0
3,100004,-1326,-408,-867.000000,-1734,421362.000000,0,0,0.0,0,...,0,0,0.0,0,0.0,0,0,0,0,0.0
4,100005,-373,-62,-190.666667,-572,26340.333333,0,0,0.0,0,...,0,0,0.0,0,0.0,0,0,0,0,0.0


In [6]:
# bur_data.to_csv('bur_data.csv',index=False)

*Right Previous Applications Data Process*

In [7]:
# Days 365.243 values -> nan
pre_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
pre_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
pre_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
pre_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
pre_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
# Add feature: value ask / value received percentage
pre_app['APP_CREDIT_PERC'] = pre_app['AMT_APPLICATION'] / pre_app['AMT_CREDIT']

app_data = process_obj(pre_app.drop('SK_ID_PREV',axis=1))
app_data = group_agg(app_data,'PRE_')
app_data.head(5)

drop col NAME_CASH_LOAN_PURPOSE :two many categories
drop col NAME_GOODS_CATEGORY :two many categories
drop col NAME_SELLER_INDUSTRY :two many categories
drop col PRODUCT_COMBINATION :two many categories


,SK_ID_CURR,PRE_AMT_ANNUITY_MIN,PRE_AMT_ANNUITY_MAX,PRE_AMT_ANNUITY_MEAN,PRE_AMT_ANNUITY_SUM,PRE_AMT_ANNUITY_VAR,PRE_AMT_APPLICATION_MIN,PRE_AMT_APPLICATION_MAX,PRE_AMT_APPLICATION_MEAN,PRE_AMT_APPLICATION_SUM,...,PRE_NAME_YIELD_GROUP_middle_MIN,PRE_NAME_YIELD_GROUP_middle_MAX,PRE_NAME_YIELD_GROUP_middle_MEAN,PRE_NAME_YIELD_GROUP_middle_SUM,PRE_NAME_YIELD_GROUP_middle_VAR,PRE_NAME_YIELD_GROUP_nan_MIN,PRE_NAME_YIELD_GROUP_nan_MAX,PRE_NAME_YIELD_GROUP_nan_MEAN,PRE_NAME_YIELD_GROUP_nan_SUM,PRE_NAME_YIELD_GROUP_nan_VAR
0,100001,3951.000,3951.000,3951.000,3951.000,NaN,24835.5,24835.5,24835.50,24835.5,...,0,0,0.000000,0,NaN,0,0,0,0,NaN
1,100002,9251.775,9251.775,9251.775,9251.775,NaN,179055.0,179055.0,179055.00,179055.0,...,0,0,0.000000,0,NaN,0,0,0,0,NaN
2,100003,6737.310,98356.995,56553.990,169661.970,2.146706e+09,68809.5,900000.0,435436.50,1306309.5,...,0,1,0.666667,2,0.333333,0,0,0,0,0.0
3,100004,5357.250,5357.250,5357.250,5357.250,NaN,24282.0,24282.0,24282.00,24282.0,...,1,1,1.000000,1,NaN,0,0,0,0,NaN
4,100005,4813.200,4813.200,4813.200,4813.200,NaN,0.0,44617.5,22308.75,44617.5,...,0,0,0.000000,0,0.000000,0,0,0,0,0.0


In [8]:
# app_data.to_csv('app_data.csv',index=False)

In [9]:
pos_data = process_obj(pre_pos.drop('SK_ID_PREV',axis=1))
pos_data = group_agg(pos_data,'POS_')
pos_data.head(5)

,SK_ID_CURR,POS_MONTHS_BALANCE_MIN,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SUM,POS_MONTHS_BALANCE_VAR,POS_CNT_INSTALMENT_MIN,POS_CNT_INSTALMENT_MAX,POS_CNT_INSTALMENT_MEAN,POS_CNT_INSTALMENT_SUM,...,POS_NAME_CONTRACT_STATUS_XNA_MIN,POS_NAME_CONTRACT_STATUS_XNA_MAX,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_XNA_SUM,POS_NAME_CONTRACT_STATUS_XNA_VAR,POS_NAME_CONTRACT_STATUS_nan_MIN,POS_NAME_CONTRACT_STATUS_nan_MAX,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_NAME_CONTRACT_STATUS_nan_SUM,POS_NAME_CONTRACT_STATUS_nan_VAR
0,100001,-96,-53,-72.555556,-653,435.277778,4.0,4.0,4.000000,36.0,...,0,0,0.0,0,0.0,0,0,0,0,0.0
1,100002,-19,-1,-10.000000,-190,31.666667,24.0,24.0,24.000000,456.0,...,0,0,0.0,0,0.0,0,0,0,0,0.0
2,100003,-77,-18,-43.785714,-1226,607.137566,6.0,12.0,10.107143,283.0,...,0,0,0.0,0,0.0,0,0,0,0,0.0
3,100004,-27,-24,-25.500000,-102,1.666667,3.0,4.0,3.750000,15.0,...,0,0,0.0,0,0.0,0,0,0,0,0.0
4,100005,-25,-15,-20.000000,-220,11.000000,9.0,12.0,11.700000,117.0,...,0,0,0.0,0,0.0,0,0,0,0,0.0


In [10]:
# pos_data.to_csv('pos_data.csv',index=False)

In [11]:
# Percentage and difference paid in each installment (amount paid and installment value)
pre_inst['PAYMENT_PERC'] = pre_inst['AMT_PAYMENT'] / pre_inst['AMT_INSTALMENT']
pre_inst['PAYMENT_DIFF'] = pre_inst['AMT_INSTALMENT'] - pre_inst['AMT_PAYMENT']
# Days past due and days before due (no negative values)
pre_inst['DPD'] = pre_inst['DAYS_ENTRY_PAYMENT'] - pre_inst['DAYS_INSTALMENT']
pre_inst['DBD'] = pre_inst['DAYS_INSTALMENT'] - pre_inst['DAYS_ENTRY_PAYMENT']
pre_inst['DPD'] = pre_inst['DPD'].apply(lambda x: x if x > 0 else 0)
pre_inst['DBD'] = pre_inst['DBD'].apply(lambda x: x if x > 0 else 0)

inst_data = process_obj(pre_inst.drop('SK_ID_PREV',axis=1))
inst_data = group_agg(inst_data,'INST_')
inst_data.head(5)

,SK_ID_CURR,INST_NUM_INSTALMENT_VERSION_MIN,INST_NUM_INSTALMENT_VERSION_MAX,INST_NUM_INSTALMENT_VERSION_MEAN,INST_NUM_INSTALMENT_VERSION_SUM,INST_NUM_INSTALMENT_VERSION_VAR,INST_NUM_INSTALMENT_NUMBER_MIN,INST_NUM_INSTALMENT_NUMBER_MAX,INST_NUM_INSTALMENT_NUMBER_MEAN,INST_NUM_INSTALMENT_NUMBER_SUM,...,INST_DPD_MIN,INST_DPD_MAX,INST_DPD_MEAN,INST_DPD_SUM,INST_DPD_VAR,INST_DBD_MIN,INST_DBD_MAX,INST_DBD_MEAN,INST_DBD_SUM,INST_DBD_VAR
0,100001,1.0,2.0,1.142857,8.0,0.142857,1,4,2.714286,19,...,0.0,11.0,1.571429,11.0,17.285714,0.0,36.0,8.857143,62.0,164.142857
1,100002,1.0,2.0,1.052632,20.0,0.052632,1,19,10.000000,190,...,0.0,0.0,0.000000,0.0,0.000000,12.0,31.0,20.421053,388.0,24.257310
2,100003,1.0,2.0,1.040000,26.0,0.040000,1,12,5.080000,127,...,0.0,0.0,0.000000,0.0,0.000000,1.0,14.0,7.160000,179.0,13.890000
3,100004,1.0,2.0,1.333333,4.0,0.333333,1,3,2.000000,6,...,0.0,0.0,0.000000,0.0,0.000000,3.0,11.0,7.666667,23.0,17.333333
4,100005,1.0,2.0,1.111111,10.0,0.111111,1,9,5.000000,45,...,0.0,1.0,0.111111,1.0,0.111111,0.0,37.0,23.666667,213.0,176.500000


In [12]:
# inst_data.to_csv('inst_data.csv',index=False)

In [13]:
card_data = process_obj(pre_card.drop('SK_ID_PREV',axis=1))
card_data = group_agg(card_data,'CARD_')
card_data.head(5)

,SK_ID_CURR,CARD_MONTHS_BALANCE_MIN,CARD_MONTHS_BALANCE_MAX,CARD_MONTHS_BALANCE_MEAN,CARD_MONTHS_BALANCE_SUM,CARD_MONTHS_BALANCE_VAR,CARD_AMT_BALANCE_MIN,CARD_AMT_BALANCE_MAX,CARD_AMT_BALANCE_MEAN,CARD_AMT_BALANCE_SUM,...,CARD_NAME_CONTRACT_STATUS_Signed_MIN,CARD_NAME_CONTRACT_STATUS_Signed_MAX,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_SUM,CARD_NAME_CONTRACT_STATUS_Signed_VAR,CARD_NAME_CONTRACT_STATUS_nan_MIN,CARD_NAME_CONTRACT_STATUS_nan_MAX,CARD_NAME_CONTRACT_STATUS_nan_MEAN,CARD_NAME_CONTRACT_STATUS_nan_SUM,CARD_NAME_CONTRACT_STATUS_nan_VAR
0,100006,-6,-1,-3.5,-21,3.5,0.0,0.00,0.000000,0.000,...,0,0,0.0,0,0.0,0,0,0,0,0.0
1,100011,-75,-2,-38.5,-2849,462.5,0.0,189000.00,54482.111149,4031676.225,...,0,0,0.0,0,0.0,0,0,0,0,0.0
2,100013,-96,-1,-48.5,-4656,776.0,0.0,161420.22,18159.919219,1743352.245,...,0,0,0.0,0,0.0,0,0,0,0,0.0
3,100021,-18,-2,-10.0,-170,25.5,0.0,0.00,0.000000,0.000,...,0,0,0.0,0,0.0,0,0,0,0,0.0
4,100023,-11,-4,-7.5,-60,6.0,0.0,0.00,0.000000,0.000,...,0,0,0.0,0,0.0,0,0,0,0,0.0


In [14]:
# card_data.to_csv('card_data.csv',index=False)

**Join Processed Data to Train and Test**

In [15]:
# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None):
    # Read data and merge
#     df = pd.read_csv('../input/application_train.csv', nrows= num_rows)
#     test_df = pd.read_csv('../input/application_test.csv', nrows= num_rows)
    df = pd.read_csv('application_train.csv', nrows= num_rows)
    test_df = pd.read_csv('application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    
    # Process categorical features
    df = process_obj(df)
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    # Some new features (ratios)
    df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
    df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
    df['NEW_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())
    df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
    df['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    
    df['NEW_OBS_DEF_30_RATIO'] = (df['OBS_30_CNT_SOCIAL_CIRCLE'] - df['DEF_30_CNT_SOCIAL_CIRCLE']) / (df['OBS_30_CNT_SOCIAL_CIRCLE']+1)
    df['NEW_OBS_DEF_60_RATIO'] = (df['OBS_60_CNT_SOCIAL_CIRCLE'] - df['DEF_60_CNT_SOCIAL_CIRCLE']) / (df['OBS_60_CNT_SOCIAL_CIRCLE']+1)
    df['YEARS_LIVING_LENGTH'] = (df['YEARS_BEGINEXPLUATATION_AVG'] - df['YEARS_BUILD_AVG']) / df['YEARS_BUILD_AVG']
    df['SHARED_COMMONAREA'] = df['COMMONAREA_AVG'] / (df['COMMONAREA_AVG'] + df['APARTMENTS_AVG'] + df['BASEMENTAREA_AVG'])
    
    return df.drop('index',axis=1)

In [16]:
data = application_train_test()
data.head(5)

Train samples: 307511, test samples: 48744
drop col OCCUPATION_TYPE :two many categories
drop col ORGANIZATION_TYPE :two many categories


,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,NEW_SCORES_STD,NEW_CAR_TO_BIRTH_RATIO,NEW_CAR_TO_EMPLOY_RATIO,NEW_PHONE_TO_BIRTH_RATIO,NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER,NEW_CREDIT_TO_INCOME_RATIO,NEW_OBS_DEF_30_RATIO,NEW_OBS_DEF_60_RATIO,YEARS_LIVING_LENGTH,SHARED_COMMONAREA
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.092026,NaN,NaN,0.119860,1.780220,2.007889,0.000000,0.000000,0.570090,0.188406
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219895,NaN,NaN,0.049389,0.696970,4.790750,0.500000,0.500000,0.237563,0.289059
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.122792,-0.001365,-0.115556,0.042791,3.622222,2.000000,0.000000,0.000000,NaN,NaN
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.151008,NaN,NaN,0.032465,0.203027,2.316167,0.666667,0.666667,NaN,NaN
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.151008,NaN,NaN,0.055489,0.364055,4.222222,0.000000,0.000000,NaN,NaN


In [17]:
data_frame = [data,bur_data,app_data,pos_data,inst_data,card_data]

for df in data_frame:
    df = df.set_index('SK_ID_CURR')
    
df = pd.concat(data_frame,axis=1,join_axes=[data.index])
df.head(5) 

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,CARD_NAME_CONTRACT_STATUS_Signed_MIN,CARD_NAME_CONTRACT_STATUS_Signed_MAX,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_SUM,CARD_NAME_CONTRACT_STATUS_Signed_VAR,CARD_NAME_CONTRACT_STATUS_nan_MIN,CARD_NAME_CONTRACT_STATUS_nan_MAX,CARD_NAME_CONTRACT_STATUS_nan_MEAN,CARD_NAME_CONTRACT_STATUS_nan_SUM,CARD_NAME_CONTRACT_STATUS_nan_VAR
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df.shape

(356255, 1083)

In [19]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]

In [20]:
# train_df.to_csv('train.csv',index=False)
# test_df.to_csv('test.csv',index=False)